In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [22]:
df = pd.read_csv("pol_only.csv", low_memory = False)


In [23]:
df.head(5)

,Ticket Number,Violation Date,Violation Time,Issuing Agency,Respondent First Name,Respondent Last Name,Balance Due,Violation Location (Borough),Violation Location (Block No.),Violation Location (Lot No.),...,Charge #8: Code Description,Charge #8: Infraction Amount,Charge #9: Code,Charge #9: Code Section,Charge #9: Code Description,Charge #9: Infraction Amount,Charge #10: Code,Charge #10: Code Section,Charge #10: Code Description,Charge #10: Infraction Amount
0,0198948842,02/13/2020,03:15:00,POLICE DEPARTMENT,JUAN,FLORES,112.0,QUEENS,1769.0,55.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0198984253,02/12/2020,22:45:00,POLICE DEPARTMENT,ERIC A,NICASIO,112.0,QUEENS,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0199274121,02/10/2020,23:15:00,POLICE DEPARTMENT,ELIAS,SULUSA,25.0,QUEENS,1482.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0199009773,02/11/2020,19:33:00,POLICE DEPARTMENT,YOON S,CHO,0.0,QUEENS,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0199417525,12/30/2019,17:25:00,POLICE DEPARTMENT,STAVROULA,KOKKOROS,0.0,QUEENS,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df.shape

(766394, 78)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 766394 entries, 0 to 766393
Data columns (total 78 columns):
 #   Column                                                           Non-Null Count   Dtype  
---  ------                                                           --------------   -----  
 0   Ticket Number                                                    766394 non-null  object 
 1   Violation Date                                                   763596 non-null  object 
 2   Violation Time                                                   764415 non-null  object 
 3   Issuing Agency                                                   766394 non-null  object 
 4   Respondent First Name                                            498874 non-null  object 
 5   Respondent Last Name                                             762112 non-null  object 
 6   Balance Due                                                      313916 non-null  float64
 7   Violation Location (Borough) 

In [26]:
# set it to datetime to check for min and max 
df["Violation Date"] = pd.to_datetime(df["Violation Date"])

print("Oldest ticket from the dataset is on {}".format(df["Violation Date"].min()))
print("Latest ticket from the dataset is on {}".format(df["Violation Date"].max()))

Oldest ticket from the dataset is on 1900-01-01 00:00:00
Latest ticket from the dataset is on 2021-11-12 00:00:00


# Identify the target variable "Hearing Result" and drop nulls and relable. 

In [27]:
df.dropna(subset=["Hearing Result"], inplace=True)

In [28]:
df.shape

(611757, 78)

In [29]:
df["Hearing Result"].value_counts()

WRITTEN OFF     263475
DEFAULTED       120361
DISMISSED       109692
IN VIOLATION     85387
DEFAULT          21334
SETTL IN-VIO     10815
ADJOURNED          411
POP IN-VIOL        123
POP/IN-VIOL        117
NONE                24
STIPULATED          11
COSRV INVIO          7
Name: Hearing Result, dtype: int64

In [30]:
df["Issuing Agency"].value_counts()

POLICE DEPARTMENT                 289601
SANITATION POLICE                 184021
NYPD TRANSPORT INTELLIGENCE DI    102721
POLICE DEPT                        32283
SANITATION ENVIRON. POLICE          1429
DEP. POLICE                          858
SEAGATE POLICE                       844
Name: Issuing Agency, dtype: int64

# regroup the target variable into fee collected and fee not collected

In [31]:
WRITTEN OFF     263475
DEFAULTED       120361
DISMISSED       109692
IN VIOLATION     85387
DEFAULT          21334
SETTL IN-VIO     10815
ADJOURNED          411
POP IN-VIOL        123
POP/IN-VIOL        117
NONE                24
STIPULATED          11
COSRV INVIO          7

SyntaxError: invalid syntax (<ipython-input-31-87f9e985c01a>, line 1)

In [32]:
df = df.loc[(df["Hearing Result"] != "NONE") & (df["Hearing Result"] != "ADJOURNED") & (df["Hearing Result"] != "STIPULATED")]

In [33]:
df["Hearing Result"].replace({ "WRITTEN OFF": "NO FEE COLLECTED",
                               "DEFAULTED": "FEE COLLECTED",
                               "DISMISSED": "NO FEE COLLECTED",
                               "IN VIOLATION": "FEE COLLECTED",
                               "DEFAULT": "FEE COLLECTED",
                               "SETTL IN-VIO": "NO FEE COLLECTED",
                               "COSRV INVIO": "NO FEE COLLECTED",
                               "POP IN-VIOL": "NO FEE COLLECTED",
                               "POP/IN-VIOL": "NO FEE COLLECTED"
                             
                             
                             }, inplace=True)

In [34]:
df["Hearing Result"].value_counts(normalize=True)

NO FEE COLLECTED    0.628533
FEE COLLECTED       0.371467
Name: Hearing Result, dtype: float64

In [35]:
df['Hearing Result'] = df['Hearing Result'].map({'NO FEE COLLECTED': 0,
                                     'FEE COLLECTED': 1})

In [37]:
df["Hearing Result"].value_counts(normalize=True)

0    0.628533
1    0.371467
Name: Hearing Result, dtype: float64

In [36]:
df.shape

(611311, 78)

## EDA and Cleaning Columns

only want to focus on respondents whose address is in New York because I will merge this dataframe with neighborhood data later on.

In [49]:
df["Respondent Address (State Name)"].value_counts()

NEW YORK               568515
NEW JERSEY              20209
PENNSYLVANIA             1194
FLORIDA                   920
CONNECTICUT               694
MASSACHUSETTS             537
NORTH CAROLINA            358
VIRGINIA                  281
MARYLAND                  280
CALIFORNIA                238
GEORGIA                   178
ILLINOIS                  177
SOUTH CAROLINA            163
WASHINGTON                144
TEXAS                     138
MISSOURI                  111
OHIO                      108
RHODE ISLAND              107
MICHIGAN                  101
OTHER                      56
DELAWARE                   54
TENNESSEE                  45
OKLAHOMA                   42
ALABAMA                    40
LOUISIANA                  36
DIST COLUMBIA              34
NEBRASKA                   31
ARIZONA                    29
COLORADO                   28
MAINE                      25
OREGON                     23
INDIANA                    21
WISCONSIN                  20
KENTUCKY  

In [50]:
df = df.loc[(df["Respondent Address (State Name)"] == "NEW YORK")]

## drop columns that contain all null values

In [51]:
column_list = list(df.columns)

In [52]:
empty_columns = []

for x in column_list:

    if df[x].isnull().all() == True:
        empty_columns.append(x)

df.drop(empty_columns, axis = 1, inplace=True)

In [53]:
df.shape

(568515, 48)

In [54]:
df.columns

Index(['Ticket Number', 'Violation Date', 'Violation Time', 'Issuing Agency',
       'Respondent First Name', 'Respondent Last Name', 'Balance Due',
       'Violation Location (Borough)', 'Violation Location (Block No.)',
       'Violation Location (Lot No.)', 'Violation Location (House #)',
       'Violation Location (Street Name)', 'Violation Location (City)',
       'Violation Location (Zip Code)', 'Violation Location (State Name)',
       'Respondent Address (Borough)', 'Respondent Address (House #)',
       'Respondent Address (Street Name)', 'Respondent Address (City)',
       'Respondent Address (Zip Code)', 'Respondent Address (State Name)',
       'Hearing Status', 'Hearing Result', 'Scheduled Hearing Location',
       'Hearing Date', 'Hearing Time', 'Decision Location (Borough)',
       'Decision Date', 'Total Violation Amount', 'Violation Details',
       'Date Judgment Docketed',
       'Respondent Address or Facility Number(For FDNY and DOB Tickets)',
       'Penalty Impos

In [55]:
columns_to_drop = ['Violation Time','Balance Due',
       'Hearing Status','Scheduled Hearing Location',
       'Hearing Date', 'Hearing Time',
       'Decision Date', 'Total Violation Amount', 'Violation Details',
       'Date Judgment Docketed',
       'Respondent Address or Facility Number(For FDNY and DOB Tickets)',
        'Additional Penalties or Late Fees',
       'Compliance Status']

df.drop(columns_to_drop, axis = 1, inplace=True)
df.shape

(568515, 35)

In [56]:
# pd.options.display.max_colwidth = 1000000
# pd.set_option('display.max_columns', 2000000000)
# pd.set_option('display.max_rows', 1000000000)
# pd.set_option('display.expand_frame_repr', True)

### Feature enginnering the violation location and respondent's address column

In [57]:
df.dropna(subset=["Violation Location (Zip Code)"], inplace=True)
df.dropna(subset=["Respondent Address (Zip Code)"], inplace=True)

In [58]:
viol_cols = ['Violation Location (Borough)', 'Violation Location (Block No.)',
       'Violation Location (Lot No.)', 'Violation Location (House #)',
       'Violation Location (Street Name)',
       'Violation Location (City)', 'Violation Location (Zip Code)',
       'Violation Location (State Name)']


resp_cols = ['Respondent Address (Borough)',
       'Respondent Address (House #)', 'Respondent Address (Street Name)',
       'Respondent Address (City)', 'Respondent Address (Zip Code)',
       'Respondent Address (State Name)']



df['complete violation location'] = df[viol_cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
df['complete violation location'] = df['complete violation location'].map(lambda x: x.replace("nan", ''))


df['complete respondent location'] = df[resp_cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
df['complete respondent location'] = df['complete respondent location'].map(lambda x: x.replace("nan", ''))

In [59]:
print("Violation Location (Zip Code) has {} nulls".format(df['Violation Location (Zip Code)'].isnull().sum()))
print("Respondent Address (Zip Code) has {} nulls".format(df['Respondent Address (Zip Code)'].isnull().sum()))

Violation Location (Zip Code) has 0 nulls
Respondent Address (Zip Code) has 0 nulls


In [60]:
df.shape

(284325, 37)

In [61]:

df["complete violation location"].isnull().sum()

0

In [62]:
df.shape

(284325, 37)

# now that the dataframe is cut down, let's split the dataset into train and test. I will save them into separate dataframe, csv files, and not touch the testing until it it ready.

In [63]:
X = df.drop('Hearing Result', axis=1)
y = df["Hearing Result"]
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y, 
                                                    random_state=42,
                                                    stratify=y)

In [64]:
y_train.value_counts(normalize=True)

0    0.429200
1    0.273505
3    0.150786
2    0.146509
Name: Hearing Result, dtype: float64

In [65]:
df_train=pd.concat([X_train, y_train], axis=1)
df_test=pd.concat([X_test, y_test], axis=1)

In [66]:
df_train.head(1)

,Ticket Number,Violation Date,Issuing Agency,Respondent First Name,Respondent Last Name,Violation Location (Borough),Violation Location (Block No.),Violation Location (Lot No.),Violation Location (House #),Violation Location (Street Name),...,Charge #2: Code Section,Charge #2: Code Description,Charge #2: Infraction Amount,Charge #3: Code,Charge #3: Code Section,Charge #3: Code Description,Charge #3: Infraction Amount,complete violation location,complete respondent location,Hearing Result
545274,0162420262,2012-08-30,POLICE DEPARTMENT,RONALD,BARONE,BROOKLYN,NaN,NaN,1711,EAST 33 STREET,...,A.C. 24-238 A,IMPROPER AUDIBLE BURGLAR ALARM WITH NO AUTOMAT...,280.0,NaN,NaN,NaN,NaN,BROOKLYN 1711 EAST 33 STREET BROOKLYN 11234 ...,BROOKLYN 21 SEBA AVENUE BROOKLYN 11229 NEW YORK,2


In [67]:
df_test.shape

(71082, 37)

In [68]:
df_train.to_csv('df_train.csv')
df_test.to_csv('df_test.csv')